In [39]:
import numpy as np
from scipy.stats import poisson
from functools import lru_cache
from tqdm import tqdm

In [51]:
ACTIONS = np.arange(-5, 6)
MAX_CARS_LOC = 20
N_CARS_PER_LOC = (MAX_CARS_LOC + 1)    #[0, 20]
N_STATES = N_CARS_PER_LOC * N_CARS_PER_LOC
CAR_RENT_COST = 10
CAR_MOVE_COST = -2

LAMBDA_RET_LOC1 = 3
LAMBDA_RET_LOC2 = 2
LAMBDA_RNT_LOC1 = 3
LAMBDA_RNT_LOC2 = 4

MAX_POISSON_OUTCOME = 11
GAMMA = 0.9
THETA = 1e-4

#Pre-computing poisson pmf due to high computational cost
poisson_pmf_req1 = [poisson.pmf(i, LAMBDA_RNT_LOC1) for i in range(MAX_POISSON_OUTCOME)]
poisson_pmf_req2 = [poisson.pmf(i, LAMBDA_RNT_LOC2) for i in range(MAX_POISSON_OUTCOME)]
poisson_pmf_ret1 = [poisson.pmf(i, LAMBDA_RET_LOC1) for i in range(MAX_POISSON_OUTCOME)]
poisson_pmf_ret2 = [poisson.pmf(i, LAMBDA_RET_LOC2) for i in range(MAX_POISSON_OUTCOME)]

@lru_cache(maxsize=None)
def calculate_expected_val(state, action): #R(s, a) | P(s' |s, a)
    #Estado do dia anterior
    n1_end_prev, n2_end_prev = divmod(state, N_CARS_PER_LOC)

    #Inicializa recompensa e executa a ação, que é mover os carros de uma localização para outra
    #Valores positivos loc1 -> loc2
    #Valores negativos loc2 -> loc1
    expected_total_reward = 0.0

    n1_after_move = n1_end_prev - action
    n2_after_move = n2_end_prev + action

    if not (0 <= n1_after_move <= MAX_CARS_LOC) or not (0 <= n2_after_move <= MAX_CARS_LOC):
        return 0.0, np.zeros(N_STATES)

    #Inicializa a probabilidade dos estados seguintes
    next_state_prob = np.zeros(N_STATES)

    #Verifica os limites para n1 e n2 se manterem entre 0 e 20
    n1_start_day = min(max(0, n1_after_move), MAX_CARS_LOC)
    n2_start_day = min(max(0, n2_after_move), MAX_CARS_LOC)

    #Custo por mover cada carro
    move_cost = abs(action) * CAR_MOVE_COST

    #Como o processo é estocástico, é necessário verificar todas as possibilidades para encontrar a probabilidade de cair no próximo estado
    for req1 in range(MAX_POISSON_OUTCOME):
        prob_req1 = poisson_pmf_req1[req1] 
        for req2 in range(MAX_POISSON_OUTCOME):
            prob_req2 = poisson_pmf_req2[req2]
            for ret1 in range(MAX_POISSON_OUTCOME):
                prob_ret1 = poisson_pmf_ret1[ret1]
                for ret2 in range(MAX_POISSON_OUTCOME):
                    prob_ret2 = poisson_pmf_ret2[ret2]
                    

                    prob_joint_outcome = prob_req1 * prob_req2 * prob_ret1 * prob_ret2 #Probabilidade conjunta (São independentes)

                    # Calcula o número de carros alugados. Se houverem mais clientes do que carros, aluga todos os carros restantes.
                    rented_1 = min(n1_start_day, req1)
                    rented_2 = min(n2_start_day, req2)
                    
                    # Calcula a recompensa obtida do aluguel das 2 localizações menos o custo de mover os carros
                    rental_income = (rented_1 + rented_2) * CAR_RENT_COST
                    current_reward = rental_income - move_cost
                    
                    # Esperança da recompensa
                    expected_total_reward += prob_joint_outcome * current_reward

                    # Calcula os carros do próximo dia
                    cars_after_rentals_1 = n1_start_day - rented_1
                    cars_after_rentals_2 = n2_start_day - rented_2

                    # Retornam os carros do "dia anterior"
                    next_n1 = min(MAX_CARS_LOC, cars_after_rentals_1 + ret1)
                    next_n2 = min(MAX_CARS_LOC, cars_after_rentals_2 + ret2)
                    
                    #Atribui probabilidade para o próximo estado obtido
                    next_s_idx = next_n1 * N_CARS_PER_LOC + next_n2
                    next_state_prob[next_s_idx] += prob_joint_outcome

    #Normaliza as probabilidades
    sum_probs = np.sum(next_state_prob)
    if sum_probs > 0:
        next_state_prob /= sum_probs

    #Retorna a recompensa esperada de cada próximo estado R(s,a) e sua probabilidade P(s' |s, a)
    return expected_total_reward, next_state_prob

def get_prob_policy(policy, state, action):
    if action in policy[state]:
        return 1/len(policy[state])

    return 0

def policy_eval(state_values, policy, theta, gamma):
    while True:
        delta = 0
        v_old = state_values.copy()
        for state in tqdm(range(N_STATES), desc="Policy Evaluation"):
            v_new = 0
            for action in ACTIONS:
                expected_r, next_state_prob = calculate_expected_val(state, action) # R(s,a) , p(s' |s, a)
                v_action = expected_r + gamma * np.dot(next_state_prob, state_values)
                v_new += v_action * get_prob_policy(policy, state, action)

            state_values[state] = v_new

            delta = max(delta, abs(v_old[state] - v_new))

        if delta < theta:
            break


def policy_improv(policy, state_values, gamma):
    pol_stable = True
    old_policy = [i.copy() for i in policy]
    for state in tqdm(range(N_STATES), desc="Policy Improvement"):
        best_q = float("-inf")
        best_action = 0

        for action in ACTIONS:
            expected_r, next_state_prob = calculate_expected_val(state, action) # R(s,a) , p(s' |s, a)
            print(expected_r)

            if np.sum(next_state_prob) == 0:
                continue

            q_sa = 0
            for next_s in range(N_STATES):
                q_sa += next_state_prob[next_s] * (expected_r + gamma * state_values[next_s])

            if best_q < q_sa:
                best_q = q_sa
                best_action = action

        policy[state] = [best_action]
        if policy[state] != old_policy[state]:
            pol_stable = False

    return pol_stable

def policy_iter(policy, state_values, gamma):
    iterations = 0
    while True:
        iterations += 1
        print("Política atual: ")
        print(np.array(policy).reshape(21,21))
        policy_eval(state_values, policy, 1e-3, gamma)
        pol_stable = policy_improv(policy, state_values, gamma)
        if pol_stable:
            break
    print("Iterations: ", iterations)

policy = [[0] for s in range(N_STATES)]
state_values = np.zeros(N_STATES)

policy_iter(policy, state_values, 0.9)
print(np.array(policy).reshape(21,21))

Política atual: 
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


Policy Improvement:  20%|█▉        | 88/441 [00:00<00:00, 370.86it/s]

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
11.462520717719304
9.78264246049182
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
21.436118661752367
21.24516317821156
18.833093733124425
0.0
0.0
0.0
0.0
0.0
0.0
0.0
29.17633244525593
31.218761122244164
30.295614450843704
26.41916263003876
0.0
0.0
0.0
0.0
0.0
0.0
34.683162068230004
38.95897490574734
40.26921239487749
37.881683347757836
32.05272169266171
0.0
0.0
0.0
0.0
0.0
38.51495357080725
44.465804528722316
48.00942617838047
47.85528129179149
43.51524241038184
35.73377092099298
0.0
0.0
0.0
0.0
0.0
48.29759603129981
53.51625580135471
55.59549507529572
53.48884035441369
47.196291638713404
37.85281228189168
0.0
0.0
0.0
0.0
0.0
57.34804730393205
61.10232469826957
61.22905413791843
57.16988958274602
49.3153329996116
38.9305150645017
0.0
0.0
0.0
0.0
0.0
64.93411620084653
66.73588376089279
64.91010336624943
59.28893094364359
50.39303578222172
39.41316723094661
0.0
0.0
0.0
0.0
0.0
70.56767526346867
70.41693298922402
67.02914472714849
60.366633726

Policy Improvement:  36%|███▋      | 160/441 [00:00<00:00, 299.50it/s]

73.11345636885746
70.97432603299954
68.2004507907631
68.35493539116908
66.83438187117852
63.08044419065646
57.093122349606766
49.61687773486927
77.69134508176325
76.76783373622918
75.22235151902537
73.29858322721957
71.02721942110304
68.2004507907631
68.35493539116908
66.83438187117852
63.08044419065646
57.093122349606766
49.61687773486927
78.76904786437368
77.25048590267507
75.40747837738768
73.35147661532353
71.02721942110304
68.2004507907631
68.35493539116908
66.83438187117852
63.08044419065646
57.093122349606766
49.61687773486927
79.25170003081841
77.43561276103725
75.46037176549115
73.35147661532353
71.02721942110304
68.2004507907631
68.35493539116908
66.83438187117852
63.08044419065646
57.093122349606766
49.61687773486927
79.43682688918094
77.48850614914079
75.46037176549115
73.35147661532353
71.02721942110304
68.2004507907631
68.35493539116908
66.83438187117852
63.08044419065646
57.093122349606766
49.61687773486927
79.4897202772844
77.48850614914079
75.46037176549115
73.35147661

...

79.25170003081841
77.44368883768935
75.50344417430162
73.51030612281107
71.51716807131848
69.5240300198269
71.51716807131848
73.51030612281107
75.50344417430162
77.49658222579272
79.48164420063242
79.43682688918094
77.49658222579272
75.50344417430162
73.51030612281107
71.51716807131848
69.5240300198269
71.51716807131848
73.51030612281107
75.50344417430162
77.49658222579272
79.48164420063242
79.4897202772844
77.49658222579272
75.50344417430162
73.51030612281107
71.51716807131848
69.5240300198269
71.51716807131848
73.51030612281107
75.50344417430162
77.49658222579272
79.48164420063242
79.4897202772844
77.49658222579272
75.50344417430162
73.51030612281107
71.51716807131848
69.5240300198269
71.51716807131848
73.51030612281107
75.50344417430162
77.49658222579272
0.0
79.4897202772844
77.49658222579272
75.50344417430162
73.51030612281107
71.51716807131848
69.5240300198269
71.51716807131848
73.51030612281107
75.50344417430162
0.0
0.0
79.4897202772844
77.49658222579272
75.50344417430162
73.5103

Policy Improvement:  91%|█████████▏| 403/441 [00:01<00:00, 305.39it/s]

0.0
0.0
0.0
0.0
0.0
29.872842542414297
41.64862305439792
52.69221237852185
62.27141932692714
69.89811644104174
75.57230372086481
0.0
0.0
0.0
0.0
31.86598059390581
39.6554850029061
50.69907432703028
60.27828127543639
67.90497838955044
73.5791656693739
77.69134508176325
0.0
0.0
0.0
33.85911864539706
41.64862305439792
48.70593627553869
58.28514322394468
65.91184033805861
71.5860276178815
75.69820703027199
78.76904786437368
0.0
0.0
35.85225669688868
43.641761105889024
50.69907432703028
56.29200517245324
63.918702286567786
69.59288956639094
73.70506897878057
76.77590981288144
79.25170003081841
0.0
37.84539474837965
45.63489915738079
52.69221237852185
58.28514322394468
61.92556423507602
67.59975151489928
71.71193092728963
74.78277176139055
77.25856197932679
79.43682688918094
0.0
47.62803720887216
54.68535043001269
60.27828127543639
63.918702286567786
65.60661346340801
69.7187928757979
72.78963370989963
75.26542392783578
77.44368883768935
79.4897202772844
0.0
56.67848848150528
62.271419326927

Policy Improvement: 100%|██████████| 441/441 [00:01<00:00, 285.95it/s]


0.0
69.5240300198269
71.51716807131848
73.51030612281107
75.50344417430162
77.49658222579272
79.4897202772844
0.0
0.0
0.0
0.0
0.0
69.5240300198269
71.51716807131848
73.51030612281107
75.50344417430162
77.49658222579272
79.4897202772844
0.0
0.0
0.0
0.0
0.0
69.5240300198269
71.51716807131848
73.51030612281107
75.50344417430162
77.49658222579272
79.4897202772844
0.0
0.0
0.0
0.0
0.0
69.5240300198269
71.51716807131848
73.51030612281107
75.50344417430162
77.49658222579272
79.4897202772844
0.0
0.0
0.0
0.0
0.0
69.5240300198269
71.51716807131848
73.51030612281107
75.50344417430162
77.49658222579272
0.0
0.0
0.0
0.0
0.0
0.0
69.5240300198269
71.51716807131848
73.51030612281107
75.50344417430162
0.0
0.0
0.0
0.0
0.0
0.0
0.0
69.5240300198269
71.51716807131848
73.51030612281107
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
69.5240300198269
71.51716807131848
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
69.5240300198269
0.0
0.0
0.0
0.0
0.0
Iterations:  6
[[ 0 -1 -2 -3 -4 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5 -5]
 [ 1 